In [1]:
import requests
import re
from bs4 import BeautifulSoup

In [2]:
import urllib.request
import ast

In [3]:
link = urllib.request.urlopen('https://doj.gov.in/vacancy-position/')
link = link.read()
page = BeautifulSoup(link, 'lxml')

In [4]:
page

<!DOCTYPE html>
<html lang="en-US">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<meta content="Vacancy Position | Department of Justice | India" name="title"/>
<meta content="en" name="lang"/>
<title>Vacancy Position | Department of Justice | India</title>
<link href="http://gmpg.org/xfn/11" rel="profile"/>
<link href="https://doj.gov.in/xmlrpc.php" rel="pingback"/>
<link href="https://doj.gov.in/wp-content/themes/sdo-theme/css/print.css" media="print" rel="stylesheet" type="text/css"/>
<meta content="max-image-preview:large" name="robots"/>
<meta content="1. SUPREME COURT 2. HIGH COURTS 3. VACANCY POSITIONS AS ON 01.08.2024 (PDF 66KB)" name="description"/>
<meta content="Vacancy, Position, Vacancy Position" name="keywords"/>
<link href="https://doj.gov.in/wp-includes/css/dist/block-library/style.min.css?ver=6.6.1" id="wp-block-library-css" media="all" rel="stylesheet"/>
<sty

In [5]:
tag_obj = page.find_all('h2', class_ = 'heading4 box-style1 border1 accent-border-color border-top0 border-right0 pd-bottom5 pd-top5')[2]

In [6]:
pdfUrl = tag_obj.a
pdfUrl = pdfUrl.get('href')
print(pdfUrl)

https://cdnbbsr.s3waas.gov.in/s35d6646aad9bcc0be55b2c82f69750387/uploads/2024/08/20240801994943198.pdf


In [55]:
import pandas as pd

In [8]:
import tabula

In [9]:
df = tabula.convert_into(pdfUrl, "test.csv", output_format='csv', stream=True)

Failed to import jpype dependencies. Fallback to subprocess.
No module named 'jpype'
'pages' argument isn't specified.Will extract only from page 1 by default.


In [10]:
df = pd.read_csv("test.csv")

In [11]:
type(df)

pandas.core.frame.DataFrame

In [12]:
df.head()

,A.,Supreme Court,Unnamed: 2,34,Unnamed: 4,Unnamed: 5,34.1,Unnamed: 7,Unnamed: 8,0,Unnamed: 10
0,B.,High Court,Pmt.,Addl,Total,Pmt.,Addl,Total,Pmt.,Addl,Total
1,1,Allahabad,119,41,160,75,9,84,44,32,76
2,2,Andhra Pradesh,28,9,37,22,6,28,6,3,9
3,3,Bombay,71,23,94,56,10,66,15,13,28
4,4,Calcutta,54,18,72,34,10,44,20,8,28


In [13]:
col1 = df.iloc[1:,1]
col1 = pd.concat([pd.Series(['Supreme']), col1])
col1

0              Supreme
1            Allahabad
2       Andhra Pradesh
3               Bombay
4             Calcutta
5         Chhattisgarh
6                Delhi
7              Gauhati
8              Gujarat
9     Himachal Pradesh
10    J & K and Ladakh
11           Jharkhand
12           Karnataka
13              Kerala
14      Madhya Pradesh
15              Madras
16             Manipur
17           Meghalaya
18              Orissa
19               Patna
20    Punjab & Haryana
21           Rajasthan
22              Sikkim
23           Telangana
24             Tripura
25         Uttarakhand
26               Total
dtype: object

In [14]:
df.columns

Index(['A.', 'Supreme Court', 'Unnamed: 2', '34', 'Unnamed: 4', 'Unnamed: 5',
       '34.1', 'Unnamed: 7', 'Unnamed: 8', '0', 'Unnamed: 10'],
      dtype='object')

In [15]:
col2 = pd.concat([pd.Series(int(df.columns[3])), df.iloc[1:, 4]])

In [16]:
col3 = pd.concat([pd.Series([34]), df.iloc[1:, 7]])
col4 = pd.concat([pd.Series(int(df.columns[9])), df.iloc[1:, 10]])

In [17]:
dict = {'Supreme Court/High Court' : col1, 'Sanctioned Strength': col2, 'Working Strength': col3, 'Vacancy' : col4}

In [18]:
df = pd.DataFrame(dict)
df

,Supreme Court/High Court,Sanctioned Strength,Working Strength,Vacancy
0,Supreme,34,34,0
1,Allahabad,160,84,76
2,Andhra Pradesh,37,28,9
3,Bombay,94,66,28
4,Calcutta,72,44,28
5,Chhattisgarh,22,15,7
6,Delhi,60,39,21
7,Gauhati,30,24,6
8,Gujarat,52,29,23
9,Himachal Pradesh,17,12,5


In [19]:
link2 = urllib.request.urlopen('https://dashboard.doj.gov.in/sanctiondata/sanctioned_posts')
link2 = link2.read()
page2 = BeautifulSoup(link2, 'lxml')

In [20]:
tables = []
for i in range(1, 5):
    script = page2.find_all('script', type="text/javascript")[i]
    script_content = script.string
    pattern = r'google\.visualization\.arrayToDataTable\((\[\s*\[.*?\]\s*\])\);'
    # Use re.DOTALL to match across multiple lines
    match = re.search(pattern, script_content, re.DOTALL)
    arr = match.group(1)
    arr = ast.literal_eval(arr)
    tables.append(arr)

In [21]:
jr_judge_sanc = pd.DataFrame(tables[0][1:], columns=tables[0][0])
jr_judge_sanc.head()

,State,Number
0,Andaman and Nicobar,0
1,Andhra Pradesh,325
2,Arunachal Pradesh,14
3,Assam,205
4,Bihar,885


In [22]:
sr_judge_sanc = pd.DataFrame(tables[1][1:], columns=tables[1][0])
sr_judge_sanc.head()

,State,Number
0,Andaman and Nicobar,0
1,Andhra Pradesh,135
2,Arunachal Pradesh,19
3,Assam,149
4,Bihar,420


In [23]:
dist_judge_sanc = pd.DataFrame(tables[2][1:], columns=tables[2][0])
dist_judge_sanc.head()

,State,Number
0,Andaman and Nicobar,0
1,Andhra Pradesh,158
2,Arunachal Pradesh,11
3,Assam,131
4,Bihar,714


In [24]:
total_sanc = pd.DataFrame(tables[3][1:], columns=tables[3][0])
total_sanc.head()

,State,Number
0,Andaman and Nicobar,0
1,Andhra Pradesh,618
2,Arunachal Pradesh,44
3,Assam,485
4,Bihar,2019


In [11]:
link3 = urllib.request.urlopen('https://dashboard.doj.gov.in/sanctiondata/vacancy')
link3 = link3.read()
page3 = BeautifulSoup(link3, 'lxml')

In [13]:
tables = []
for i in range(1, 5):
    script = page3.find_all('script', type="text/javascript")[i]
    script_content = script.string
    pattern = r'google\.visualization\.arrayToDataTable\((\[\s*\[.*?\]\s*\])\);'
    # Use re.DOTALL to match across multiple lines
    match = re.search(pattern, script_content, re.DOTALL)
    arr = match.group(1)
    arr = ast.literal_eval(arr)
    tables.append(arr)


// Load google charts
google.charts.load('current', {'packages':['corechart']});
google.charts.setOnLoadCallback(drawChart);

// Draw the chart and set the chart values
function drawChart() {
  var data = google.visualization.arrayToDataTable([
  ['State', 'Number'],  
  ['Andaman and Nicobar', 4],
  ['Andhra Pradesh', 37],
  ['Arunachal Pradesh', 2],
  ['Assam', 27],
  ['Bihar', 254],
  ['Chandigarh', 0],
  ['Chhattisgarh', 43],
  ['D & N Haveli', 0],
  ['Daman & Diu', 0],
  ['Delhi', 53],
  ['Goa', 6],
  ['Gujarat', 191],
  ['Haryana', 240],
  ['Himachal Pradesh', 6],
  ['Jammu & Kashmir', 61],
  ['Jharkhand', 118],
  ['Karnataka', 98],
  ['Kerala', 68],
  ['Ladakh', 4],
  ['Lakshadweep', 0],
  ['Madhya Pradesh', 220],
  ['Maharashtra', 198],
  ['Manipur', 13],
  ['Meghalaya', 7],
  ['Mizoram', 18],
  ['Nagaland', 5],
  ['Odisha', 160],
  ['Puducherry', 5],
  ['Punjab', 144],
  ['Rajasthan', 121],
  ['Sikkim', 3],
  ['Tamil Nadu', 149],
  ['Telangana', 20],
  ['Tripura', 13],
  ['Ut

In [27]:
jr_judge_vac = pd.DataFrame(tables[0][1:], columns=tables[0][0])
jr_judge_vac.head()

,State,Number
0,Andaman and Nicobar,4
1,Andhra Pradesh,37
2,Arunachal Pradesh,2
3,Assam,27
4,Bihar,254


In [28]:
sr_judge_vac = pd.DataFrame(tables[1][1:], columns=tables[1][0])
sr_judge_vac.head()

,State,Number
0,Andaman and Nicobar,3
1,Andhra Pradesh,3
2,Arunachal Pradesh,6
3,Assam,10
4,Bihar,187


In [29]:
dist_judge_vac = pd.DataFrame(tables[2][1:], columns=tables[2][0])
dist_judge_vac.head()

,State,Number
0,Andaman and Nicobar,6
1,Andhra Pradesh,31
2,Arunachal Pradesh,0
3,Assam,21
4,Bihar,159


In [30]:
total_vac = pd.DataFrame(tables[3][1:], columns=tables[3][0])
total_vac.head()

,State,Number
0,Andaman and Nicobar,13
1,Andhra Pradesh,71
2,Arunachal Pradesh,8
3,Assam,58
4,Bihar,600


In [31]:
vacancy_dict = {'State': total_vac.iloc[:, 0], 'Civil Judge(Junior Division)': jr_judge_vac.iloc[:,1], 'Civil Judge(Senior Division)': sr_judge_vac.iloc[:,1], 'District Judge/DJ': dist_judge_vac.iloc[:,1], 'Total': total_vac.iloc[:,1]}
vacancy = pd.DataFrame(vacancy_dict)
vacancy.head()

,State,Civil Judge(Junior Division),Civil Judge(Senior Division),District Judge/DJ,Total
0,Andaman and Nicobar,4,3,6,13
1,Andhra Pradesh,37,3,31,71
2,Arunachal Pradesh,2,6,0,8
3,Assam,27,10,21,58
4,Bihar,254,187,159,600


In [32]:
sanctioned_dict = {'State': total_sanc.iloc[:, 0], 'Civil Judge(Junior Division)': jr_judge_sanc.iloc[:,1], 'Civil Judge(Senior Division)': sr_judge_sanc.iloc[:,1], 'District Judge/DJ': dist_judge_sanc.iloc[:,1], 'Total': total_sanc.iloc[:,1]}
sanctioned = pd.DataFrame(sanctioned_dict)
sanctioned.head()

,State,Civil Judge(Junior Division),Civil Judge(Senior Division),District Judge/DJ,Total
0,Andaman and Nicobar,0,0,0,0
1,Andhra Pradesh,325,135,158,618
2,Arunachal Pradesh,14,19,11,44
3,Assam,205,149,131,485
4,Bihar,885,420,714,2019


In [3]:
link4 = urllib.request.urlopen('https://dashboard.doj.gov.in/sanctiondata/working_strength')
link4 = link4.read()
page4 = BeautifulSoup(link4, 'lxml')

In [48]:
scripts = page4.find_all('script', type="", src="")

In [60]:
tables = []
for i in range(1, 4):
    script_content = scripts[i].string
    pattern = r'chart\.data\s*=\s*(\[(.*?)\])'
    match = re.search(pattern, script_content, re.DOTALL)
    arr = match.group(1)
    arr = arr.replace("'", '"')
    arr = re.sub(r'(\w+):', r'"\1":', arr)
    arr = arr.replace('category', 'State').replace('first', 'SC').replace('second', 'ST').replace('third', 'OBC').replace('fourth', 'Specially Abled').replace('fifth', 'Women')
    arr = ast.literal_eval(arr)
    tables.append(arr)

[[{'State': 'Andaman & Nicobar',
   'SC': 0,
   'ST': 0,
   'OBC': 0,
   'Specially Abled': 4,
   'Women': 4},
  {'State': 'Andhra Pradesh',
   'SC': 45,
   'ST': 18,
   'OBC': 118,
   'Specially Abled': 106,
   'Women': 287},
  {'State': 'Arunachal Pradesh',
   'SC': 0,
   'ST': 12,
   'OBC': 0,
   'Specially Abled': 0,
   'Women': 12},
  {'State': 'Assam',
   'SC': 15,
   'ST': 22,
   'OBC': 0,
   'Specially Abled': 141,
   'Women': 178},
  {'State': 'Bihar',
   'SC': 115,
   'ST': 9,
   'OBC': 254,
   'Specially Abled': 253,
   'Women': 631},
  {'State': 'Chandigarh',
   'SC': 3,
   'ST': 0,
   'OBC': 2,
   'Specially Abled': 13,
   'Women': 18},
  {'State': 'Chhattisgarh',
   'SC': 28,
   'ST': 64,
   'OBC': 34,
   'Specially Abled': 42,
   'Women': 168},
  {'State': 'D & N Haveli',
   'SC': 0,
   'ST': 0,
   'OBC': 1,
   'Specially Abled': 0,
   'Women': 1},
  {'State': 'Daman & Diu',
   'SC': 0,
   'ST': 0,
   'OBC': 0,
   'Specially Abled': 1,
   'Women': 1},
  {'State': 'Delhi'

In [61]:
jr_judge_working = pd.DataFrame(tables[0])
jr_judge_working.head()

,State,SC,ST,OBC,Specially Abled,Women
0,Andaman & Nicobar,0,0,0,4,4
1,Andhra Pradesh,45,18,118,106,287
2,Arunachal Pradesh,0,12,0,0,12
3,Assam,15,22,0,141,178
4,Bihar,115,9,254,253,631


In [62]:
sr_judge_working = pd.DataFrame(tables[1])
sr_judge_working.head()

,State,SC,ST,OBC,Specially Abled,Women
0,Andaman & Nicobar,0,0,0,3,3
1,Andhra Pradesh,26,10,49,46,131
2,Arunachal Pradesh,0,11,0,0,11
3,Assam,7,16,0,115,138
4,Bihar,62,5,22,144,233


In [63]:
dist_judge_working = pd.DataFrame(tables[2])
dist_judge_working.head()

,State,SC,ST,OBC,Specially Abled,Women
0,Andaman & Nicobar,0,0,0,0,0
1,Andhra Pradesh,15,1,35,0,32
2,Arunachal Pradesh,0,9,1,0,1
3,Assam,0,0,0,0,34
4,Bihar,39,1,51,1,34


In [65]:
heading_figures = page4.find_all("h3")
heading_figures

[<h3>Civil Judge(Junior Division)</h3>,
 <h3>1280</h3>,
 <h3>470</h3>,
 <h3>2064</h3>,
 <h3>215</h3>,
 <h3>3711</h3>,
 <h3>Civil Judge(Senior Division)</h3>,
 <h3>688</h3>,
 <h3>276</h3>,
 <h3>1184</h3>,
 <h3>82</h3>,
 <h3>1550</h3>,
 <h3>District Judge/DJ</h3>,
 <h3>633</h3>,
 <h3>203</h3>,
 <h3>1395</h3>,
 <h3>55</h3>,
 <h3>1435</h3>]

In [72]:
total_figures = pd.DataFrame(columns=['Judge', 'SC', 'ST', 'OBC', 'Specially Abled', 'Women'])
list_df = []
k = 0
for i in range(3):
    inner = []
    for j in range(6):
        inner.append(heading_figures[k].string)
        k += 1
    list_df.append(inner)

[['Civil Judge(Junior Division)', '1280', '470', '2064', '215', '3711'],
 ['Civil Judge(Senior Division)', '688', '276', '1184', '82', '1550'],
 ['District Judge/DJ', '633', '203', '1395', '55', '1435']]

In [74]:
total_figures = pd.DataFrame(list_df, columns=['Judge', 'SC', 'ST', 'OBC', 'Specially Abled', 'Women'])
total_figures.head()

,Judge,SC,ST,OBC,Specially Abled,Women
0,Civil Judge(Junior Division),1280,470,2064,215,3711
1,Civil Judge(Senior Division),688,276,1184,82,1550
2,District Judge/DJ,633,203,1395,55,1435
